In [ ]:
# ========================================
# 🧠 Cell 1 — Train Custom CNN Model
# ========================================
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ======================
# Transforms
# ======================
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomResizedCrop(224, scale=(0.9, 1.1)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# ======================
# Datasets & Loaders
# ======================
data_dir = "dataset"

train_ds = datasets.ImageFolder(os.path.join(data_dir, "train"), transform=train_transform)
val_ds   = datasets.ImageFolder(os.path.join(data_dir, "val"), transform=val_transform)
test_ds  = datasets.ImageFolder(os.path.join(data_dir, "test"), transform=val_transform)

batch_size = 64

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader   = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader  = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=4)

class_names = train_ds.classes
print("Classes:", class_names)

# ======================
# Model Definition
# ======================
class AntiSpoofNet(nn.Module):
    def __init__(self, num_classes=2):
        super(AntiSpoofNet, self).__init__()
        self.features = nn.Sequential(
            # block 1
            nn.Conv2d(3, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.1),
            nn.Conv2d(32, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d(2),

            # block 2
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.1),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d(2),

            # block 3
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.1),
            nn.Conv2d(128, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d(2),

            # block 4
            nn.Conv2d(128, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d(2),

            # block 5
            nn.Conv2d(256, 512, 3, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.1),
            nn.Conv2d(512, 512, 3, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d(2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(512 * 7 * 7, 512),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# ======================
# Training setup
# ======================
model = AntiSpoofNet(num_classes=2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5)

# ======================
# Training Loop
# ======================
best_val_acc = 0.0
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    
    loop = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}]")
    for imgs, labels in loop:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        loop.set_postfix(loss=loss.item(), acc=100*correct/total)

    # Validation
    model.eval()
    val_correct, val_total = 0, 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
    val_acc = 100 * val_correct / val_total
    scheduler.step(val_acc)
    print(f"Epoch {epoch+1} done. Train Acc: {100*correct/total:.2f}% | Val Acc: {val_acc:.2f}%")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_customcnn.pth")
        print("✅ Saved best model")

print("Training complete. Best Val Accuracy:", best_val_acc)

In [ ]:
# ========================================
# 🧪 Cell 2 — Test on test set
# ========================================
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

model = AntiSpoofNet(num_classes=2).to(device)
model.load_state_dict(torch.load("best_customcnn.pth", map_location=device))
model.eval()

all_labels = []
all_preds = []

with torch.no_grad():
    for imgs, labels in tqdm(test_loader, desc="Testing"):
        imgs = imgs.to(device)
        outputs = model(imgs)
        _, preds = torch.max(outputs, 1)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

cm = confusion_matrix(all_labels, all_preds)
report = classification_report(all_labels, all_preds, target_names=class_names, output_dict=True)

real_acc = report["real"]["recall"] * 100
spoof_acc = report["spoof"]["recall"] * 100
overall_acc = (np.trace(cm) / np.sum(cm)) * 100

print(f"\n✅ Overall Accuracy: {overall_acc:.2f}%")
print(f"🎭 Real Accuracy: {real_acc:.2f}%")
print(f"🕵️ Spoof Accuracy: {spoof_acc:.2f}%")

# Plot confusion matrix
plt.figure(figsize=(5, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix (Test Set)")
plt.show()

In [ ]:
# ========================================
# 🚀 Cell 3 — Evaluate on Custom Folder
# ========================================
custom_dir = "custom_folder"  # <-- Your custom unseen dataset folder

custom_ds = datasets.ImageFolder(custom_dir, transform=val_transform)
custom_loader = DataLoader(custom_ds, batch_size=batch_size, shuffle=False, num_workers=4)
print("Custom classes:", custom_ds.classes)

all_labels = []
all_preds = []

model.eval()
with torch.no_grad():
    for imgs, labels in tqdm(custom_loader, desc="Evaluating Custom Folder"):
        imgs = imgs.to(device)
        outputs = model(imgs)
        _, preds = torch.max(outputs, 1)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

cm_custom = confusion_matrix(all_labels, all_preds)
report_custom = classification_report(all_labels, all_preds, target_names=custom_ds.classes, output_dict=True)

real_acc = report_custom["real"]["recall"] * 100
spoof_acc = report_custom["spoof"]["recall"] * 100
overall_acc = (np.trace(cm_custom) / np.sum(cm_custom)) * 100

print(f"\n🧾 Custom Folder Evaluation Results:")
print(f"✅ Overall Accuracy: {overall_acc:.2f}%")
print(f"🎭 Real Accuracy: {real_acc:.2f}%")
print(f"🕵️ Spoof Accuracy: {spoof_acc:.2f}%")

plt.figure(figsize=(5, 4))
sns.heatmap(cm_custom, annot=True, fmt='d', cmap='Greens',
            xticklabels=custom_ds.classes, yticklabels=custom_ds.classes)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix (Custom Folder)")
plt.show()